### **ETL** - *extract tranform load*

In [1]:
import os
import pandas as pd

In [2]:
DATA_FOLDER = 'data'
BIKE_RENTALS_FILE = 'tripdata_connected.csv'
WEATHER_FILE = 'weather_data.csv'

bike_rentals_df = pd.read_csv(os.path.join('..', DATA_FOLDER, BIKE_RENTALS_FILE))
weather_df = pd.read_csv(os.path.join('..', DATA_FOLDER, WEATHER_FILE),
                         encoding="Windows-1250", sep=";")

In [3]:
bike_rentals_df.head()

,Ride_Id,Rideable_Type,Started_At,Ended_At,Start_Station_Name,Start_Station_Id,End_Station_Name,End_Station_Id,Start_Lat,Start_Lng,End_Lat,End_Lng,Member_Casual,Ride_Length,Day_Of_The_Week,Day
0,0BE9C131A5705D92,classic_bike,10/1/2021 0:00,10/1/2021 0:05,Damen Ave & Cortland St,13133,Winchester Ave & Elston Ave,KA1504000140,41.915983,-87.677335,41.924091,-87.676460,casual,0:05:00,6,Friday
1,ABA2BDC3595FC3E9,classic_bike,10/1/2021 0:00,10/1/2021 0:10,Morgan St & Lake St,TA1306000015,Noble St & Milwaukee Ave,13290,41.885483,-87.652305,41.900680,-87.662600,casual,0:10:00,6,Friday
2,74483AC18C8C6B90,classic_bike,10/1/2021 0:00,10/1/2021 0:08,Halsted St & Roscoe St,TA1309000025,Greenview Ave & Diversey Pkwy,13294,41.943670,-87.648950,41.932590,-87.665936,casual,0:08:00,6,Friday
3,A7711BCB74523614,electric_bike,10/1/2021 0:02,10/1/2021 0:03,NaN,NaN,NaN,NaN,41.930000,-87.700000,41.930000,-87.700000,casual,0:01:00,6,Friday
4,B0B9EB7622461EF4,classic_bike,10/1/2021 0:02,10/1/2021 0:17,MLK Jr Dr & 29th St,TA1307000139,Clinton St & Roosevelt Rd,WL-008,41.842052,-87.617000,41.867118,-87.641088,member,0:15:00,6,Friday


### **wypożyczenie**

Tabela wypożyczenie, środek schematu gwiazdy.

In [18]:
df_rentals = bike_rentals_df[['Ride_Id', 'Started_At', 'Ended_At', 'Start_Station_Id', 'End_Station_Name']].copy()

df_rentals['Started_At'] = pd.to_datetime(df_rentals['Started_At'], format='%m/%d/%Y %H:%M')
df_rentals['Ended_At'] = pd.to_datetime(df_rentals['Ended_At'], format='%m/%d/%Y %H:%M')
df_rentals['Date'] = df_rentals['Started_At'].dt.date
df_rentals = df_rentals[['Ride_Id', 'Date', 'Start_Station_Id', 'Start_Station_Id']]
df_rentals.dropna(inplace=True)
df_rentals.head()

,Ride_Id,Date,Start_Station_Id,Start_Station_Id
0,0BE9C131A5705D92,2021-10-01,13133,13133
1,ABA2BDC3595FC3E9,2021-10-01,TA1306000015,TA1306000015
2,74483AC18C8C6B90,2021-10-01,TA1309000025,TA1309000025
4,B0B9EB7622461EF4,2021-10-01,TA1307000139,TA1307000139
5,BC372D36022271AC,2021-10-01,TA1306000015,TA1306000015


### **czas**

In [28]:
df_time = bike_rentals_df[['Ride_Id', 'Started_At', 'Ride_Length', 'Day']].copy()


# df_time['Duration'] = (df_time['Ended_At'] - df_time['Started_At']).dt.total_seconds() // 60
df_time['Ride_Length'] = pd.to_datetime(df_time['Ride_Length'], format='%H:%M:%S', errors='coerce')
df_time.dropna(inplace=True)
df_time['Ride_Length'].dt.minute.astype(int)
df_time['Time_Of_The_Day'] = df_time['Started_At'].dt.hour


def remap_time(time):
    if time >= 5 and time < 12:
        return 'Morning'
    elif time >= 12 and time < 18:
        return 'Afternoon'
    elif time >= 18 and time < 22:
        return 'Evening'
    else:
        return 'Night'
    
 
df_time['Time_Of_The_Day'] = df_time['Started_At']
df_time['Time_Of_The_Day'] = df_time['Time_Of_The_Day'].dt.hour.apply(remap_time)
display(df_time)
df_time.head()
df_time = df_time[['Ride_Id', 'Date', 'Duration', 'Day']].copy()
df_time.head()

KeyError: 'Started_At'

### **lokalizacja**

In [7]:
bike_rentals_df.columns

Index(['Ride_Id', 'Rideable_Type', 'Started_At', 'Ended_At',
       'Start_Station_Name', 'Start_Station_Id', 'End_Station_Name',
       'End_Station_Id', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng',
       'Member_Casual', 'Ride_Length', 'Day_Of_The_Week', 'Day'],
      dtype='object')

In [14]:
start_df = bike_rentals_df[['Ride_Id', 'Start_Station_Id', 'Start_Station_Name', 'Start_Lat', 'Start_Lng']].copy()
start_df.columns = ['Ride_Id', 'Station_Id', 'Station_Name', 'Lat', 'Lng']

end_df = bike_rentals_df[['Ride_Id', 'End_Station_Id', 'End_Station_Name', 'End_Lat', 'End_Lng']].copy()
end_df.columns = ['Ride_Id', 'Station_Id', 'Station_Name', 'Lat', 'Lng']

df_location = pd.concat([start_df, end_df])
df_location.head()

,Ride_Id,Station_Id,Station_Name,Lat,Lng
0,0BE9C131A5705D92,13133,Damen Ave & Cortland St,41.915983,-87.677335
1,ABA2BDC3595FC3E9,TA1306000015,Morgan St & Lake St,41.885483,-87.652305
2,74483AC18C8C6B90,TA1309000025,Halsted St & Roscoe St,41.943670,-87.648950
3,A7711BCB74523614,NaN,NaN,41.930000,-87.700000
4,B0B9EB7622461EF4,TA1307000139,MLK Jr Dr & 29th St,41.842052,-87.617000


In [15]:
len(df_location)

10277426

In [16]:
df_location = df_location.drop_duplicates(subset=['Station_Id'])
df_location.dropna(inplace=True)
df_location.head()

,Ride_Id,Station_Id,Station_Name,Lat,Lng
0,0BE9C131A5705D92,13133,Damen Ave & Cortland St,41.915983,-87.677335
1,ABA2BDC3595FC3E9,TA1306000015,Morgan St & Lake St,41.885483,-87.652305
2,74483AC18C8C6B90,TA1309000025,Halsted St & Roscoe St,41.943670,-87.648950
4,B0B9EB7622461EF4,TA1307000139,MLK Jr Dr & 29th St,41.842052,-87.617000
6,F8DBF095F01E1B68,TA1308000022,Pine Grove Ave & Irving Park Rd,41.954383,-87.648043


In [17]:
len(df_location)

1299

### **typy**

In [10]:
df_types = bike_rentals_df[['Ride_Id', 'Rideable_Type', 'Member_Casual']].copy()
df_types.head()

,Ride_Id,Rideable_Type,Member_Casual
0,0BE9C131A5705D92,classic_bike,casual
1,ABA2BDC3595FC3E9,classic_bike,casual
2,74483AC18C8C6B90,classic_bike,casual
3,A7711BCB74523614,electric_bike,casual
4,B0B9EB7622461EF4,classic_bike,member


### **pogoda**

In [11]:
weather_df.head()

,time,temperature_2m (°C),precipitation (mm),cloudcover (%),windspeed_10m (km/h),latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,2021-10-01T00:00,"19,3",0,23.0,"9,4","41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT
1,2021-10-01T01:00,"18,8",0,14.0,"9,9","41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT
2,2021-10-01T02:00,"18,5",0,4.0,"9,6","41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT
3,2021-10-01T03:00,"18,1",0,0.0,"9,7","41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT
4,2021-10-01T04:00,"17,8",0,0.0,10,"41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT


In [12]:
df_weather = pd.DataFrame()
weather_cols = ['Time', 'Temperature', 'Precipitation', 'Wind_Speed', 'Cloudcover', 'Latitude', 'Longitude']
df_weather[weather_cols] = weather_df[[
    'time', 'temperature_2m (°C)', 'precipitation (mm)', 'windspeed_10m (km/h)', 'cloudcover (%)', 'latitude', 'longitude'
    ]].copy()

In [13]:
df_weather['Time'] = pd.to_datetime(df_weather['Time'], format='%Y-%m-%d %H:%M:%S')

float_cols = ['Temperature', 'Precipitation', 'Wind_Speed', 'Cloudcover', 'Latitude', 'Longitude']
df_weather[float_cols] = df_weather[float_cols].replace(',', '.', regex=True).astype(float)

df_weather['Date'] = df_weather['Time'].dt.date

df_weather.head(8)

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,19.3,0.0,9.4,23.0,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,18.8,0.0,9.9,14.0,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,18.5,0.0,9.6,4.0,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,18.1,0.0,9.7,0.0,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,17.8,0.0,10.0,0.0,41.800003,-87.6,2021-10-01
5,2021-10-01 05:00:00,17.5,0.0,10.2,0.0,41.800003,-87.6,2021-10-01
6,2021-10-01 06:00:00,17.3,0.0,10.7,0.0,41.800003,-87.6,2021-10-01
7,2021-10-01 07:00:00,17.1,0.0,10.5,0.0,41.800003,-87.6,2021-10-01


In [14]:
def remap_perciption(perciption):
    if perciption == 0:
        return 'No'
    elif perciption < 2.5:
        return 'Light'
    elif perciption < 7.6:
        return 'Moderate'
    elif perciption < 50.8:
        return 'Heavy'
    else:
        return 'Violent'
    
df_weather['Precipitation'] = df_weather['Precipitation'].apply(remap_perciption)
df_weather.head()

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,19.3,No,9.4,23.0,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,18.8,No,9.9,14.0,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,18.5,No,9.6,4.0,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,18.1,No,9.7,0.0,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,17.8,No,10.0,0.0,41.800003,-87.6,2021-10-01


In [15]:
def remap_wind(wind):
    if wind < 1.6:
        return 'Calm'
    elif wind < 5.5:
        return 'Light'
    elif wind < 11.1:
        return 'Moderate'
    elif wind < 19.8:
        return 'Fresh'
    elif wind < 28.5:
        return 'Strong'
    elif wind < 38.9:
        return 'Near Gale'
    elif wind < 49.6:
        return 'Gale'
    elif wind < 61.2:
        return 'Strong Gale'
    elif wind < 74.2:
        return 'Storm'
    else:
        return 'Violent Storm'

df_weather['Wind_Speed'] = df_weather['Wind_Speed'].apply(remap_wind)
df_weather.head()

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,19.3,No,Moderate,23.0,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,18.8,No,Moderate,14.0,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,18.5,No,Moderate,4.0,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,18.1,No,Moderate,0.0,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,17.8,No,Moderate,0.0,41.800003,-87.6,2021-10-01


In [16]:
df_weather['Wind_Speed'].value_counts()

Fresh            3668
Moderate         2391
Strong           1768
Light             496
Near Gale         351
Gale               46
Calm               40
Violent Storm       3
Name: Wind_Speed, dtype: int64

In [17]:
def remap_temperature(temperature):
    if temperature < -20:
        return 'Extreme Cold'
    elif temperature < -10:
        return 'Very Cold'
    elif temperature < 0:
        return 'Cold'
    elif temperature < 10:
        return 'Cool'
    elif temperature < 20:
        return 'Mild'
    elif temperature < 30:
        return 'Warm'
    elif temperature < 40:
        return 'Hot'
    else:
        return 'Very Hot'

df_weather['Temperature'] = df_weather['Temperature'].apply(remap_temperature)
df_weather.head()

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,Mild,No,Moderate,23.0,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,Mild,No,Moderate,14.0,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,Mild,No,Moderate,4.0,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,Mild,No,Moderate,0.0,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,Mild,No,Moderate,0.0,41.800003,-87.6,2021-10-01


In [18]:
df_weather['Temperature'].value_counts()

Cool         2614
Mild         2334
Warm         2277
Cold         1316
Very Cold     189
Hot            30
Very Hot        3
Name: Temperature, dtype: int64

In [19]:
def remap_cloudcover(cloud_cover):
    if cloud_cover == 0:
        return 'No'
    elif cloud_cover > 0 and cloud_cover < 10:
        return 'Few'
    elif cloud_cover >=10 and cloud_cover < 25:
        return 'Isolated'
    elif cloud_cover >=25 and cloud_cover < 50:
        return 'Scattered'
    elif cloud_cover >=50 and cloud_cover < 90:
        return 'Broken'
    else:  
        return 'Overcast'


df_weather['Cloudcover'] = df_weather['Cloudcover'].apply(remap_cloudcover)
df_weather.head()

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,Mild,No,Moderate,Isolated,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,Mild,No,Moderate,Isolated,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,Mild,No,Moderate,Few,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,Mild,No,Moderate,No,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,Mild,No,Moderate,No,41.800003,-87.6,2021-10-01
